In [1]:
import pandas as pd
import pipeline as p
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns

/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
import cleaning as c

In [3]:
pd.set_option('mode.chained_assignment', None)

In [4]:
df = p.open_pkl('breast_2000-2015_df.pkl')

In [5]:
# col_descr_dict = p.open_pkl('col_descr_dict.pkl')
# col_lookup = {v:k for k, v in col_descr_dict.items()}

In [5]:
df3 = c.clean_df2(df)

In [6]:
df3.head()

,PUBCSNUM,REG,MAR_STAT,RACE1V,NHIADE,SEX,AGE_DX,YR_BRTH,SEQ_NUM,MDXRECMP,...,N_VALUE,M_VALUE,MALIGCOUNT,BENBORDCOUNT,TARGET,TUMSIZ,SURG,MAR_STAT_MOD,RACE_MOD,INVAS
0,92500005,0000001544,2,01,0,2,45.0,1962,0,01,...,,,01,00,1,40.0,1.0,1.0,3,1
1,92500010,0000001544,1,02,0,2,49.0,1958,0,02,...,,,01,00,1,50.0,1.0,0.0,3,1
2,92500017,0000001544,4,01,0,2,65.0,1942,1,01,...,,,02,00,1,NaN,1.0,0.0,3,0
4,92500024,0000001544,2,01,0,2,69.0,1938,0,01,...,,,01,00,1,15.0,1.0,1.0,3,1
5,92500032,0000001544,2,02,0,2,65.0,1942,1,01,...,,,02,00,1,34.0,1.0,1.0,3,1


**Stage**

Dervied AJCC-6
- 00: Stage 0
- 10: Stage I
- 32: Stage IIA
- 33: Stage IIB
- 51-53: Stage III
- 70: Stage IV
- 88: N/A

In [9]:
p.vcs(df3,'DAJCCSTG')

      93839
10    75420
32    38971
33    17185
00    14401
52    11573
99     6214
54     5463
70     3412
53     3169
51      176
88      128
Name: DAJCCSTG, dtype: int64

In [16]:
def convert_stage_dajcc(x):
    if (x == '00'):
        return 0
    elif x == '10':
        return 1
    elif (x == '32') | (x == '33'):
        return 2
    elif (x == '51') | (x == '52') | (x == '53'):
        return 3
    elif x == '70':
        return 4
    else:
        return None

In [17]:
df3['DAJCCSTG'] = df3['DAJCCSTG'].apply(lambda x: convert_stage_dajcc(x))

In [18]:
p.vcs(df3,'DAJCCSTG')

1.0    75420
2.0    56156
3.0    14918
0.0    14401
4.0     3412
Name: DAJCCSTG, dtype: int64

DERIVED SS2000

- 0         In Situ
- 1       Localized
- 2 Regional, direct extension
- 3 Regional, lymph nodes only
- 4 Regional, extension and nodes 5   Regional, NOS
- 7       Distant
- 8 Not applicable
- 9             Unknown/Unstaged

In [7]:
p.vcs(df3,'DSS2000S')

1    100707
      93839
3     44678
0     14247
4      7411
7      3649
2      2758
9      2662
Name: DSS2000S, dtype: int64

SUMM2K: SEER Summary Stage 2000

- 0 In situ
- 1 Localized only
- 2 Regional by direct extension only
- 3 Regional lymph nodes involved only
- 4 Regional by both direct extension and lymph
- 5 Regional, NOS
- 7 Distant site(s)/node(s) involved
- 9 Unknown/unstaged/unspecified/DCO

In [13]:
p.vcs(df3,'SUMM2K')

1    153775
2     86587
0     19373
7      5198
9      5018
Name: SUMM2K, dtype: int64

HST_STGA: SEER Historic Stage A. A simplified version of stage
- 0 In Situ (non-invasive)
- 1 Localized (invasive)
- 2 Regional
- 4 Distant 
- 9 Unstaged

In [10]:
p.vcs(df3,'HST_STGA')

1    153774
2     84051
0     19373
4      8045
9      4708
Name: HST_STGA, dtype: int64

In [19]:
df3[['DAJCCSTG','DSS2000S','SUMM2K','HST_STGA']].sample(20)

,DAJCCSTG,DSS2000S,SUMM2K,HST_STGA
367221,1.0,1,1,1
344154,2.0,1,1,1
109012,NaN,,0,0
469854,NaN,,0,0
322308,NaN,,1,1
173468,2.0,1,1,1
318596,NaN,,1,1
452753,NaN,9,9,9
405962,NaN,,2,2
179429,1.0,1,1,1


In [21]:
len(df3[df3['SUMM2K'] != df3['HST_STGA']])

8356

In [24]:
df3[df3['SUMM2K'] != df3['HST_STGA']][['SUMM2K','HST_STGA']].sample(10)

,SUMM2K,HST_STGA
146496,2,4
538062,7,4
494844,7,4
397307,7,4
148806,7,4
428403,2,4
400739,2,4
69981,7,4
324767,2,4
82670,7,4


Use Historic Stage